In [1]:
import numpy as np
import pandas as pd
from IPython.display import HTML
import os
import openai
import sys
from dotenv import load_dotenv


sys.path.append('../..')
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
# Progress bar
def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

# read the repo in PATH and append the texts in a list
def get_data(PATH):
    list_dir = os.listdir(PATH)
    texts = []
    fund_names = []
    out = display(progress(0, len(list_dir)-1), display_id=True)
    for ii, filename in enumerate(list_dir) :
        with open(PATH+'/'+filename, 'r', encoding="utf8") as f :
            txt = f.read()
            try :
                txt_split = txt.split('<head_breaker>')
                summary = txt_split[1].strip()
                fund_name = txt_split[0].strip()
            except :
                summary = txt
                fund_name = ''
        texts.append(summary)
        fund_names.append(fund_name)
        out.update(progress(ii, len(list_dir)-1))
    return fund_names, texts

In [3]:
# Get the summaries
SUMMARY_PATH = 'NLP_app/MutualFundSummary'
fund_names, summaries = get_data(SUMMARY_PATH)

In [4]:
SUMMARY_LABELS_PATH = 'NLP_app/MutualFundLabels.csv'
df_extraction = pd.DataFrame({'fund_name' : fund_names})
df_label = pd.read_csv(SUMMARY_LABELS_PATH)
df = df_label.merge(df_extraction, on='fund_name', how='inner')
target_strategies = [
    "Balanced Fund (Low Risk)",
    "Fixed Income Long Only (Low Risk)",
    "Equity Long Only (Low Risk)"
]
df  = df[df['Investment Strategy'].isin(target_strategies)]
df['Investment Strategy'].value_counts()
df['Predicted Investment Strategy'] = np.nan
df['Investment Strategy Evidence'] = np.nan

## Ask which investment strategy as fund uses

In [5]:
from langchain.document_loaders import DirectoryLoader

# Load documents from a directory
loader = DirectoryLoader(path="./NLP_app/MutualFundSummary", glob="*.txt")

# Load the text data from the directory
documents = loader.load()

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider insta

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_chroma import Chroma
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()
persist_directory = './docs/chroma'

llm_name = "gpt-4o-mini"

from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)
from langchain.chains import RetrievalQA

/var/folders/jx/vhdxb7ks06710zlsb29bkz000000gn/T/ipykernel_37138/1399366674.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()
/var/folders/jx/vhdxb7ks06710zlsb29bkz000000gn/T/ipykernel_37138/1399366674.py:11: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name=llm_name, temperature=0)


In [7]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    separators=["\n\n", "\n", ". ", " "]
)

In [8]:
from langchain.prompts import PromptTemplate

In [9]:
# # Build prompt
# template = """Use the following pieces of context to answer the question at the end. Please answer in "Balanced Fund (Low Risk)",
#     "Fixed Income Long Only (Low Risk)", "Equity Long Only (Low Risk)". Give me a confidence score from 1 to 10.
# {context}
# Question: {question}
# Helpful Answer:"""
# QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [10]:
# qa_chain = RetrievalQA.from_chain_type(
#   llm,
#   retriever=vectordb.as_retriever(),
#   return_source_documents=True,
#   chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
# )

In [11]:
df_RAG = pd.DataFrame(fund_names, columns=['fund_name'])
df_RAG['Predicted Investment Strategy'] = np.nan
df_RAG['Prediction Evidence'] = np.nan
df_RAG['Prediction Meta Data'] = np.nan

In [12]:
for ii in range(len(df_RAG)):
  print(ii)
  fund_name = df_RAG.iloc[ii]['fund_name']
  print(fund_name)

  splits=r_splitter.split_documents([documents[ii]])
  persist_directory = f"./docs/chroma/{fund_name}"
  vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
  )
  vectordb.persist()

  # Build prompt
  template = """Use the following pieces of context to answer the question at the end. Please answer in "Balanced Fund (Low Risk)",
  "Fixed Income Long Only (Low Risk)", "Equity Long Only (Low Risk)". Give me a confidence score from 1 to 10.
  {context}
  Question: {question}
  Helpful Answer:"""
  QA_CHAIN_PROMPT = PromptTemplate.from_template(template)
  qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
  )

  question = f'{fund_name} uses which of the following investment strategies (Respond with just the investment strategy): Balanced Fund (Low Risk), Fixed Income Long Only (Low Risk), Equity Long Only (Low Risk)? '
  result = qa_chain({"query": question})
  df_RAG.iloc[ii, 1] = result["result"].split('\n')[0].strip()
  df_RAG.iloc[ii, 2] = result["source_documents"][0].page_content
  df_RAG.iloc[ii, 3] = str(result["source_documents"][0].metadata)


0
Thrivent Partner Healthcare Portfolio


/var/folders/jx/vhdxb7ks06710zlsb29bkz000000gn/T/ipykernel_37138/1211873590.py:13: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()
/var/folders/jx/vhdxb7ks06710zlsb29bkz000000gn/T/ipykernel_37138/1211873590.py:30: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})
/var/folders/jx/vhdxb7ks06710zlsb29bkz000000gn/T/ipykernel_37138/1211873590.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Equity Long Only (Low Risk)' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_RAG.iloc[ii, 1] = result["result"].split('\n')[0].strip()
/var/folders/jx/vhdxb7ks06710zlsb29bkz000000gn/T/ipykernel_37138/1211873590.py:32: FutureWarn

1
Federated MDT All Cap Core Fund
2
Eaton Vance Growth Fund
3
Columbia Large Cap Growth Fund III
4
Columbia Variable Portfolio - High Yield Bond Fund
5
Columbia Dividend Opportunity Fund
6
AB Virginia Portfolio
7
NT INTERNATIONAL SMALL-MID CAP FUND
8
Dreyfus Select Managers Long/Short Fund
9
Managed Risk Growth Fund
10
Bond Fund
11
Columbia Variable Portfolio - Mid Cap Growth Fund
12
Ivy Natural Resources Fund
13
Franklin FTSE Hong Kong ETF
14
AB Real Estate Investment Portfolio
15
Eaton Vance Dividend Builder Fund
16
JPMorgan Europe Dynamic Fund
17
American Funds 2020 Target Date Retirement Fund
18
Eaton Vance Arizona Municipal Income Fund
19
Franklin LibertyQ Global Dividend ETF
20
Fidelity Simplicity RMD 2010 Fund
21
Fidelity Contrafund K6
22
Fidelity Managed Retirement 2010 Fund
23
Fidelity Advisor Technology Fund
24
CALIFORNIA TAX-FREE MONEY MARKET FUND
25
NT INTERNATIONAL VALUE FUND
26
BBH Core Select
27
T. Rowe Price Emerging Europe Fund
28
T. Rowe Price Institutional Internatio

In [13]:
df_RAG = pd.merge(df_RAG, df[['fund_name', 'Investment Strategy']], on='fund_name')

In [14]:
df_RAG['Correct?'] = df_RAG['Investment Strategy'] == df_RAG['Predicted Investment Strategy']
df_RAG.head()

,fund_name,Predicted Investment Strategy,Prediction Evidence,Prediction Meta Data,Investment Strategy,Correct?
0,Thrivent Partner Healthcare Portfolio,Equity Long Only (Low Risk),Thrivent Partner Healthcare Portfolio\n\n<head...,{'source': 'NLP_app/MutualFundSummary/Thrivent...,Equity Long Only (Low Risk),True
1,Federated MDT All Cap Core Fund,Equity Long Only (Low Risk),Federated MDT All Cap Core Fund\n\n<head_break...,{'source': 'NLP_app/MutualFundSummary/Federate...,Equity Long Only (Low Risk),True
2,Eaton Vance Growth Fund,Equity Long Only (Low Risk),Eaton Vance Growth Fund\n\n<head_breaker>\n\nE...,{'source': 'NLP_app/MutualFundSummary/Eaton Va...,Equity Long Only (Low Risk),True
3,Columbia Large Cap Growth Fund III,Equity Long Only (Low Risk),Columbia Large Cap Growth Fund III <head_breaker>,{'source': 'NLP_app/MutualFundSummary/Columbia...,Equity Long Only (Low Risk),True
4,Columbia Variable Portfolio - High Yield Bond ...,Fixed Income Long Only (Low Risk),Columbia Variable Portfolio\n\nHigh Yield Bond...,{'source': 'NLP_app/MutualFundSummary/Columbia...,Fixed Income Long Only (Low Risk),True


In [15]:
from sklearn.metrics import confusion_matrix

In [16]:
df2 = df_RAG.dropna(subset = ['Investment Strategy', 'Predicted Investment Strategy'])

In [17]:
df2['Investment Strategy'].value_counts()

Investment Strategy
Equity Long Only (Low Risk)          247
Fixed Income Long Only (Low Risk)    130
Balanced Fund (Low Risk)              84
Name: count, dtype: int64

In [18]:
df2['Predicted Investment Strategy'].value_counts()

Predicted Investment Strategy
Equity Long Only (Low Risk)          243
Fixed Income Long Only (Low Risk)    124
Balanced Fund (Low Risk)              94
Name: count, dtype: int64

In [19]:
cm = confusion_matrix(df2['Investment Strategy'], df2['Predicted Investment Strategy'])
print(cm)
# Top Row (Actual): Balanced Fund (Low Risk)
# Middle Row (Actual): Equity Long Only (Low Risk)
# Bottom Row (Actual): Fixed Income Long Only (Low Risk)
# Left Column (Predicted): Balanced Fund (Low Risk)
# Middle Column (Predicted): Equity Long Only (Low Risk)
# Right Column (Predicted): Fixed Income Long Only (Low Risk)

[[ 79   3   2]
 [ 14 232   1]
 [  1   8 121]]


In [20]:
df_RAG[df_RAG['Correct?'] == False]

,fund_name,Predicted Investment Strategy,Prediction Evidence,Prediction Meta Data,Investment Strategy,Correct?
7,Managed Risk Growth Fund,Balanced Fund (Low Risk),Managed Risk Growth Fund\n\n<head_breaker>\n\n...,{'source': 'NLP_app/MutualFundSummary/Managed ...,Equity Long Only (Low Risk),False
27,Growth-Income Fund,Balanced Fund (Low Risk),"Issuer risks — The prices of, and the income g...",{'source': 'NLP_app/MutualFundSummary/Growth-I...,Equity Long Only (Low Risk),False
34,Janus Henderson Global Research Portfolio,Equity Long Only (Low Risk),Janus Henderson Global Research Portfolio <hea...,{'source': 'NLP_app/MutualFundSummary/Janus He...,Fixed Income Long Only (Low Risk),False
58,AllianzGI Income & Growth Fund,Balanced Fund (Low Risk),AllianzGI Income & Growth Fund <head_breaker>\...,{'source': 'NLP_app/MutualFundSummary/AllianzG...,Equity Long Only (Low Risk),False
59,Multi-Manager Value Strategies Fund,Balanced Fund (Low Risk),Multi\n\nManager Value Strategies Fund\n\n<hea...,{'source': 'NLP_app/MutualFundSummary/Multi-Ma...,Equity Long Only (Low Risk),False
60,Managed Risk International Fund,Balanced Fund (Low Risk),Managed Risk International Fund\n\n<head_break...,{'source': 'NLP_app/MutualFundSummary/Managed ...,Equity Long Only (Low Risk),False
81,Global Growth and Income Fund,Balanced Fund (Low Risk),Investing in growth-oriented stocks — Growth-o...,{'source': 'NLP_app/MutualFundSummary/Global G...,Equity Long Only (Low Risk),False
87,Portfolio Series - American Funds Managed Risk...,Balanced Fund (Low Risk),Portfolio Series - American Funds Managed Risk...,{'source': 'NLP_app/MutualFundSummary/Portfoli...,Equity Long Only (Low Risk),False
108,High Income Fund,Fixed Income Long Only (Low Risk),High Income Fund\n\n<head_breaker>,{'source': 'NLP_app/MutualFundSummary/High Inc...,Balanced Fund (Low Risk),False
113,American Funds Retirement Income Portfolio - C...,Fixed Income Long Only (Low Risk),American Funds Retirement Income Portfolio - C...,{'source': 'NLP_app/MutualFundSummary/American...,Balanced Fund (Low Risk),False


In [21]:
df_RAG.to_csv("rag_predictions.csv", index=False)